In [1]:
import requests
import json
import romkan
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm

In [72]:

# 设置 Chrome 浏览器选项
# options = Options()
# options.add_argument("--headless")  # 无头模式（不打开浏览器窗口）
# options.add_argument("--disable-blink-features=AutomationControlled")  # 伪装为真实用户
# options.add_argument(
#    "--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
# )
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
# options.add_argument('--disable-gpu')
# options.add_argument('--remote-debugging-port=9222')

# 启动浏览器
# service = Service(ChromeDriverManager().install())
# driver = webdriver.Chrome(service=service, options=options)

driver = webdriver.Chrome()

search_urls = [
    "https://home.gamer.com.tw/creationDetail.php?sn=4349325",
    "https://home.gamer.com.tw/creationDetail.php?sn=4726074",
    "https://home.gamer.com.tw/creationDetail.php?sn=5944078",
    "https://home.gamer.com.tw/creationDetail.php?sn=5426900",
    "https://home.gamer.com.tw/creationDetail.php?sn=4831500",
    "https://home.gamer.com.tw/creationDetail.php?sn=6016243",
    "https://home.gamer.com.tw/creationDetail.php?sn=5941347",
    "https://home.gamer.com.tw/creationDetail.php?sn=5639854",
    "https://home.gamer.com.tw/creationDetail.php?sn=5545288",
    "https://home.gamer.com.tw/creationDetail.php?sn=6074598",
]

song_data = []

for url in tqdm(search_urls, desc="爬取中..."):
    try:
        # 访问网页
        driver.get(url)

        # 等待特定元素加载完成（例如，等待 class 为 'home_box' 的 div 加载）
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "ST1"))
        )

        # 获取 HTML
        html = driver.page_source

        # 解析 HTML
        soup = BeautifulSoup(html, "html.parser")
        song_data.append(soup)

    except Exception as e:
        print(f"访问 {url} 时出错：{e}")

# 关闭浏览器
driver.quit()

爬取中...:   0%|          | 0/10 [00:00<?, ?it/s]

In [74]:
len(song_data)

10

In [76]:
def extract_json(soup):
    lyrics_jp = []
    lyrics_hiragana = []
    lyrics_zh = []
    
    # target_div = soup.find('div', class_='home_box')
    
    # if target_div:
    bold_tags = soup.find_all('b')
    # else:
    #    raise RuntimeError("No <div> with class 'home_box' found.")
    
    filtered_bold_tags = [b for b in bold_tags if not (len(b.contents) == 1 and b.contents[0].name == 'br')]
    filtered_bold_tags2 = []
    for i in range(len(filtered_bold_tags)):
        b_tag = filtered_bold_tags[i]
        # Check if the <b> tag contains a <font> tag
        if '—————' in b_tag.get_text(strip=True):
            break
        if b_tag.find('font'):
            # If it's not the first tag, append its text to the previous <b> tag
            if i > 0:
                previous_b_tag = filtered_bold_tags2[-1]
                previous_b_tag.append(b_tag.get_text())
                filtered_bold_tags2[-1] = previous_b_tag
        else:
            filtered_bold_tags2.append(b_tag)
    
    
    for i, b in enumerate(filtered_bold_tags2):
        text_content = b.get_text(strip=True)
        if i == 0:
            title = text_content
        elif (i-1)%3 == 0:
            lyrics_jp.append(text_content)
        elif (i-1)%3 == 1:
            hiragana_text = romkan.to_hiragana(text_content)
            lyrics_hiragana.append(hiragana_text)
        else:
            lyrics_zh.append(text_content)    
    # 提取作词、作曲、编曲、演唱者
    info = {}
    for div in soup.find_all("div"):
        text = div.get_text(strip=True)
        if "作詞：" in text:
            info["lyricist"] = text.replace("作詞：", "").strip()
        if "作曲：" in text:
            info["composer"] = text.replace("作曲：", "").strip()
        if "編曲：" in text:
            info["arranger"] = text.replace("編曲：", "").strip()
        if "歌：" in text:
            info["singer"] = text.replace("歌：", "").strip()
    
    # 组织数据结构
    song_json_data = {
        "song": {
            "title": title,
            "lyrics_info": info,
            "lyrics": {
                "japanese": lyrics_jp,
                "hiragana": lyrics_hiragana,
                "chinese": lyrics_zh
            }
        }
    }

    return song_json_data

In [2]:
import re
from collections import defaultdict

def turn_txt_to_json(file, order, title, hiragana=False, info=None):
    lyrics = defaultdict(list)
    jp, zh, hira = order
    base = 4 if hiragana else 3
    with open(file, "r", encoding="utf-8") as file:
        i = 0
        for line in file:
            remove_chars = "!@#$%^&*()-+"
            clean_text = line.translate(str.maketrans("", "", remove_chars)).replace("\n", "").replace("\t", "").replace("\u3000", "")
            clean_text = re.sub(r"\s+", " ", clean_text).strip()
            ind = (i % base)
            if len(clean_text) > 0:
                if (ind in order):
                    if base == 3 and (ind == hira):
                        clean_text = romkan.to_hiragana(clean_text)
                    lyrics[ind].append(clean_text)
                i += 1
                
    lyrics_jp = lyrics[jp]
    lyrics_zh = lyrics[zh]
    lyrics_hira = lyrics[hira]
    song_json_data = {
        "song": {
            "title": title,
            "lyrics_info": info,
            "lyrics": {
                "japanese": lyrics_jp,
                "hiragana": lyrics_hira,
                "chinese": lyrics_zh
            }
        }
    }
    return song_json_data

In [3]:
file = "assets/lyrics/liar.txt"
order = (0,2,1)
title = "嘘月"
song_json = turn_txt_to_json(file, order, title, hiragana=False, info=None)

In [4]:
song_json

{'song': {'title': '嘘月',
  'lyrics_info': None,
  'lyrics': {'japanese': ['雨が降った花が散った',
    'ただ染まった頬を想った',
    '僕はずっとバケツ一杯の月光を呑んでる',
    '本当なんだ夜みたいで',
    '薄く透明な口触りで',
    'そうなんだ、って笑ってもいいけど',
    '僕は君を待っている',
    '夏が去った街は静か',
    '僕はやっと部屋に戻って',
    '夜になった',
    'こんな良い月を一人で見てる',
    '本当なんだ、昔の僕は涙が宝石で出来てたんだ',
    'そうなんだ、って笑ってもいいけど',
    '声はもうとっくに忘れた',
    '思い出も愛も死んだ',
    '風のない海辺を歩いたあの夏へ',
    '僕はさよならが欲しいんだ',
    'ただ微睡むような',
    '物一つさえ云わないまま',
    '僕は君を待っている',
    '歳を取った一つ取った',
    '何も無い部屋で春になった',
    '僕は愛を、底が抜けた柄杓で呑んでる',
    '本当なんだ味もしなくて',
    '飲めば飲むほど喉が乾いて',
    'そうなんだって笑ってもいいけど',
    '僕は夜を待っている',
    '君の鼻歌が欲しいんだ',
    'ただ微睡むような',
    '物一つさえ云わないまま',
    '僕は君を待っている',
    '君の目を覚えていない',
    '君の口を描いていない',
    '物一つさえ云わないまま',
    '僕は君を待っていない',
    '君の鼻を知っていない',
    '君の頬を思っていない',
    'さよならすら云わないまま',
    '君は夜になって行く'],
   'hiragana': ['あめ が ふった はな が ちった',
    'ただ そまった ほほ を おもった',
    'ぼく わ ずっと ばけつ いっぱい の げっこう を のんでる',
    'ほんとうなんだ よる みだいで',
    'うすく とうめいな くちざわり で',
    'そうなんだ って わらっても いい けど',
  

In [6]:
import json

file_path = "test.json"

def merge_json(json_list, file_name):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            old_dict = json.load(file)  # 读取已有 JSON 数据
    except FileNotFoundError:
        old_dict = {"song":[]}  # 如果文件不存在，创建一个空列表
    # 使用列表推导式提取所有 'song' 键对应的值

    for json_data in json_list:
        song = json_data["song"]
        old_dict["song"].append(song)

    # 保存为 JSON 文件
    with open(file_name, "w", encoding="utf-8") as json_file:
        json.dump(old_dict, json_file, ensure_ascii=False, indent=4)
    
    print("歌词数据已成功提取，并存储为 JSON 文件！")

In [7]:
merge_json([song_json], file_path)

歌词数据已成功提取，并存储为 JSON 文件！
